In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('Clean_Dataset.csv')

In [4]:
df.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [5]:
df.drop(['Unnamed: 0','flight','duration','arrival_time'],axis=1,inplace=True)

In [6]:
df.head()

,airline,source_city,departure_time,stops,destination_city,class,days_left,price
0,SpiceJet,Delhi,Evening,zero,Mumbai,Economy,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Mumbai,Economy,1,5953
2,AirAsia,Delhi,Early_Morning,zero,Mumbai,Economy,1,5956
3,Vistara,Delhi,Morning,zero,Mumbai,Economy,1,5955
4,Vistara,Delhi,Morning,zero,Mumbai,Economy,1,5955


In [7]:
df.shape

(300153, 8)

In [8]:
df.departure_time.value_counts()

Morning          71146
Early_Morning    66790
Evening          65102
Night            48015
Afternoon        47794
Late_Night        1306
Name: departure_time, dtype: int64

In [9]:
df.airline.value_counts()

Vistara      127859
Air_India     80892
Indigo        43120
GO_FIRST      23173
AirAsia       16098
SpiceJet       9011
Name: airline, dtype: int64

In [10]:
df.source_city.value_counts()

Delhi        61343
Mumbai       60896
Bangalore    52061
Kolkata      46347
Hyderabad    40806
Chennai      38700
Name: source_city, dtype: int64

In [11]:
df.destination_city.value_counts()

Mumbai       59097
Delhi        57360
Bangalore    51068
Kolkata      49534
Hyderabad    42726
Chennai      40368
Name: destination_city, dtype: int64

In [12]:
df['class'].value_counts()


Economy     206666
Business     93487
Name: class, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   airline           300153 non-null  object
 1   source_city       300153 non-null  object
 2   departure_time    300153 non-null  object
 3   stops             300153 non-null  object
 4   destination_city  300153 non-null  object
 5   class             300153 non-null  object
 6   days_left         300153 non-null  int64 
 7   price             300153 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 18.3+ MB


In [14]:
# Splitting data into train and test
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['price']),
                                                 df['price'],
                                                 test_size=0.2,
                                                random_state=42)

In [15]:
X_train.head()

,airline,source_city,departure_time,stops,destination_city,class,days_left
148417,Air_India,Kolkata,Afternoon,one,Chennai,Economy,6
36879,Indigo,Delhi,Afternoon,two_or_more,Chennai,Economy,13
274531,Air_India,Kolkata,Afternoon,one,Chennai,Business,44
166397,Vistara,Hyderabad,Night,one,Bangalore,Economy,11
272722,Vistara,Kolkata,Night,one,Chennai,Business,5


In [16]:
y_train.head()

148417    13524
36879      9940
274531    55983
166397     7927
272722    55502
Name: price, dtype: int64

In [17]:
trf1 = ColumnTransformer([
    ('oe_departure_time',OrdinalEncoder(categories=[['Early_Morning','Morning','Afternoon','Evening','Night','Late_Night']]),[2]),
    ('oe_stops',OrdinalEncoder(categories=[['zero','one','two_or_more']]),[3])
    ],remainder='passthrough')

In [18]:
trf2 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,4,5])    
])

In [19]:
trf3 = ColumnTransformer([
    ('scale',StandardScaler(),slice(0,23))
])

In [20]:
# train the model
trf4 = DecisionTreeRegressor()

In [21]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)
    ])

In [22]:
from sklearn import set_config
set_config(display='diagram')

In [23]:
pipe.fit(X_train,y_train)

C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oe_departure_time',
                                                  OrdinalEncoder(categories=[['Early_Morning',
                                                                              'Morning',
                                                                              'Afternoon',
                                                                              'Evening',
                                                                              'Night',
                                                                              'Late_Night']]),
                                                  [2]),
                                                 ('oe_stops',
                                                  OrdinalEncoder(categories=[['zero',
                                                                              'one',
                                                                              'two_or_more']]),
                                                  [3])])),
                ('trf2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 4, 5])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 23, None))])),
                ('trf4', DecisionTreeRegressor())])

In [24]:
y_pred = pipe.predict(X_test)

In [25]:
from sklearn.metrics import r2_score,mean_squared_error
r2_score(y_test,y_pred)

0.9222120380933037

In [26]:
np.sqrt(mean_squared_error(y_test,y_pred))

6332.323069770897

In [27]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [28]:
pipe.score(X_train,y_train)

0.923336645424844

In [29]:
pipe.score(X_test,y_test)

0.9222120380933037

In [30]:
parameters={
            "trf4__max_depth" : [1,3,5,7],
           "trf4__min_samples_leaf":[2,3,4,5],
           "trf4__min_weight_fraction_leaf":[0.1,0.3,0.5,0.7],
           "trf4__max_features":["auto","sqrt",None],
           "trf4__max_leaf_nodes":[None,10,30,50,70] }

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
grid=GridSearchCV(pipe,param_grid=parameters,scoring='neg_mean_squared_error',cv=3,verbose=3)

In [33]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 960 candidates, totalling 2880 fits


C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


[CV 1/3] END trf4__max_depth=1, trf4__max_features=auto, trf4__max_leaf_nodes=None, trf4__min_samples_leaf=2, trf4__min_weight_fraction_leaf=0.1;, score=-61254047.871 total time=   0.5s


C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


[CV 2/3] END trf4__max_depth=1, trf4__max_features=auto, trf4__max_leaf_nodes=None, trf4__min_samples_leaf=2, trf4__min_weight_fraction_leaf=0.1;, score=-62820102.333 total time=   0.5s


...

[CV 2/3] END trf4__max_depth=7, trf4__max_features=None, trf4__max_leaf_nodes=70, trf4__min_samples_leaf=5, trf4__min_weight_fraction_leaf=0.7;, score=nan total time=   0.3s


C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 3/3] END trf4__max_depth=7, trf4__max_features=None, trf4__max_leaf_nodes=70, trf4__min_samples_leaf=5, trf4__min_weight_fraction_leaf=0.7;, score=nan total time=   0.3s


C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
720 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\Users\Akshant\anaconda3

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('oe_departure_time',
                                                                         OrdinalEncoder(categories=[['Early_Morning',
                                                                                                     'Morning',
                                                                                                     'Afternoon',
                                                                                                     'Evening',
                                                                                                     'Night',
                                                                                                     'Late_Night']]),
                                                                         [2]),
                                                                        ('oe_stops',
                                                                         OrdinalEncoder(categories=[['zero',
                                                                                                     'one',
                                                                                                     'two_or_more']]),
                                                                         [3])])),
                                       ('trf2',
                                        ColumnTransformer(transformers...
                                        ColumnTransformer(transformers=[('scale',
                                                                         StandardScaler(),
                                                                         slice(0, 23, None))])),
                                       ('trf4', DecisionTreeRegressor())]),
             param_grid={'trf4__max_depth': [1, 3, 5, 7],
                         'trf4__max_features': ['auto', 'sqrt', None],
                         'trf4__max_leaf_nodes': [None, 10, 30, 50, 70],
                         'trf4__min_samples_leaf': [2, 3, 4, 5],
                         'trf4__min_weight_fraction_leaf': [0.1, 0.3, 0.5,
                                                            0.7]},
             scoring='neg_mean_squared_error', verbose=3)

In [35]:
grid.best_params_

{'trf4__max_depth': 5,
 'trf4__max_features': 'sqrt',
 'trf4__max_leaf_nodes': 70,
 'trf4__min_samples_leaf': 5,
 'trf4__min_weight_fraction_leaf': 0.1}

In [36]:
grid.best_score_

-59758020.397085674

In [39]:
trf5 = DecisionTreeRegressor(max_depth=5,max_features='sqrt',max_leaf_nodes=70,min_samples_leaf=5,min_weight_fraction_leaf=0.1)

In [40]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf5',trf5)
    ])

In [41]:
pipe.fit(X_train,y_train)

C:\Users\Akshant\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oe_departure_time',
                                                  OrdinalEncoder(categories=[['Early_Morning',
                                                                              'Morning',
                                                                              'Afternoon',
                                                                              'Evening',
                                                                              'Night',
                                                                              'Late_Night']]),
                                                  [2]),
                                                 ('oe_stops',
                                                  OrdinalEncoder(categories=[['zero',
                                                                              'one',
                                                                              'two_or_more']]),
                                                  [3])])),
                ('trf2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 4, 5])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 23, None))])),
                ('trf5',
                 DecisionTreeRegressor(max_depth=5, max_features='sqrt',
                                       max_leaf_nodes=70, min_samples_leaf=5,
                                       min_weight_fraction_leaf=0.1))])

In [42]:
y_pred = pipe.predict(X_test)

In [43]:
from sklearn.metrics import r2_score,mean_squared_error
r2_score(y_test,y_pred)

0.7636171802316631

In [44]:
# export 
import pickle
pickle.dump(pipe,open('pipe1.pkl','wb'))